In [38]:
from scipy.stats import norm
import numpy as np
from typing import Mapping, Tuple
from numpy.polynomial.laguerre import lagval
from scipy.integrate import trapz
from scipy.interpolate import splrep, BSpline

### Implement standard binary tree/grid-based numerical algorithm for American Option Pricing and ensure it validates against Black-Scholes formula for Europeans

In [39]:
class BinaryTreeAmerican:  
    '''
    standard binary tree numerical algorithm for American Option Pricing 
    '''
    def __init__(self, typ: str, spot: float, strike: float, T: int, t: int, r: float, sigma: float):
        self.call = True if typ == 'call' else False
        self.spot = spot
        self.strike = strike
        self.t = t
        self.T = T # expiration
        self.r = r # interest rate
        self.sigma = sigma
        self.value = self.get_value()     
        
    def get_value(self):        
        u = np.exp(self.sigma * np.sqrt(self.t / self.T))
        d = np.exp(-self.sigma * np.sqrt(self.t / self.T))
        p = (np.exp(self.r / self.T)-d) / (u-d) 
        value = np.zeros((self.T+1,self.T+1))
        value[0,0] = self.spot
        for i in range(1,self.T+1):
            value[i,0] = value[i-1,0]*u
            for j in range(1,i+1):
                value[i,j] = value[i-1,j-1]*d
        option_value = np.zeros((self.T+1,self.T+1))
        for j in range(self.T+1):
            if self.call:
                option_value[self.T, j] = max(0, value[self.T, j] - self.strike)
            else:
                option_value[self.T, j] = max(0, self.strike - value[self.T, j])
        for i in range(self.T)[::-1]: 
            for j in range(i+1):
                if self.call:
                    option_value[i,j] = max(0, value[i,j] - self.strike, np.exp(-self.r*(self.t/self.T))*(p*option_value[i+1,j]+(1-p)*option_value[i+1,j+1]))
                else:
                    option_value[i,j] = max(0, self.strike - value[i,j], np.exp(-self.r*(self.t/self.T))*(p*option_value[i+1,j]+(1-p)*option_value[i+1,j+1]))
        return option_value[0,0]

In [40]:
class EuroPricing:
    '''
    Black-Scholes pricing for Europeans
    '''
    def __init__(self, typ: str, spot: float, strike: float, T: float, t: float, r: float, sigma: float):
        self.call = True if typ == 'call' else False
        self.spot = spot
        self.strike = strike
        self.T = T 
        self.t = t
        self.r = r 
        self.sigma = sigma
        self.gap = self.T - self.t
        self.price = self.get_price()

    def get_d1_d2(self):
        temp = self.sigma * np.sqrt(self.gap)
        d1 = (np.log(self.spot/self.strike) + (self.r + self.sigma**2/2.)*self.gap)/temp
        d2 = d1 - temp
        return d1, d2

    def get_price(self):
        d1, d2 = self.get_d1_d2()
        if self.call:
            return self.spot*norm.cdf(d1) - self.strike*np.exp(-self.r*self.gap)*norm.cdf(d2)
        else:
            return self.strike*np.exp(-self.r*self.gap)*norm.cdf(-d2) - self.spot*norm.cdf(-d1)


In [41]:
# Validate the binary tree algorithm against Black-Scholes formula for Europeans
optionBT = BinaryTreeAmerican(typ="call", spot=100.0, strike=120.0, T=3, t=1.7, r=0.05, sigma=0.2)
print(optionBT.get_value())
optionBS = EuroPricing(typ="call", spot=100.0, strike=120.0, T=3, t=1.7, r=0.05, sigma=0.2)
print(optionBS.get_price())

4.736427908210924
4.660152144720065


In [46]:
class GridAmerican(EuroPricing):
    '''
    Grid_based numerical algorithm for American Option Pricing 
    '''
    def __init__(self, typ: str, spot: float, strike: float, T: float, t: float, r: float, sigma: float,
                num_dt: int, num_dx: int):
        super().__init__(typ, spot, strike, T, t, r, sigma)
        self.payoff = lambda x: (1.0 if self.call else -1.) * (x - self.strike)
        self.ir = lambda tt: self.r * tt  # interest rate with time
        self.isig = lambda tt: (self.sigma ** 2) * tt # volatility with time
        m, v = self.get_price_mean_var(self.spot, 0., self.T)
        self.get_price = self.get_price_grid(num_dt, num_dx, m, np.sqrt(v)*4)
        
    def get_price_mean_var(self, x: float, t: float, dt: float):
        # get the mean and variance of future price given the current price 
        ir_t = self.ir(t)
        ir_t1 = self.ir(t + dt)
        isig_t = self.isig(t)
        isig_t1 = self.isig(t + dt)
        ir_diff = ir_t1 - ir_t
        isig_diff = isig_t1 - isig_t
        mean = np.log(x) + ir_diff - isig_diff / 2.
        var = isig_diff
        return mean, var

    def get_price_grid(self, num_dt: int, num_dx: int, center: float, width: float):
        dt = self.T / num_dt
        x_pts = 2 * num_dx + 1
        lsp = np.linspace(center - width, center + width, x_pts)
        prices = np.exp(lsp) 
        res = np.empty([num_dt, x_pts])
        res[-1, :] = [max(self.payoff(p), 0.) for p in prices]
        sample_points = 201
        for i in range(num_dt - 2, -1, -1):
            t = (i + 1) * dt
            knots, coeffs, order = splrep(prices, res[i + 1, :], k=3)
            spline_func = BSpline(knots, coeffs, order)
            disc = np.exp(self.ir(t) - self.ir(t + dt))
            for j in range(x_pts):
                m, v = self.get_price_mean_var(prices[j], t, dt)
                stdev = np.sqrt(v)
                norm_dist = norm(loc=m, scale=stdev)

                def integr_func(x: float, spline_func=spline_func,norm_dist=norm_dist) -> float:
                    val = np.exp(x)
                    return max(spline_func(val), 0.) * norm_dist.pdf(x)

                low, high = (m - 4 * stdev, m + 4 * stdev)
                disc_exp_payoff = disc * trapz(
                    np.vectorize(integr_func)(np.linspace(low, high, sample_points)),
                    dx=(high - low) / (sample_points - 1)
                ) / (norm_dist.cdf(high) - norm_dist.cdf(low))
                res[i, j] = max(self.payoff(prices[j]), disc_exp_payoff)

        knots, coeffs, order = splrep(prices, res[0, :], k=3)
        spline_func = BSpline(knots, coeffs, order)
        disc = np.exp(-self.ir(dt))
        m, v = self.get_price_mean_var(self.spot, 0., dt)
        stdev = np.sqrt(v)
        norm_dist = norm(loc=m, scale=stdev)

        def integr_func0(x: float, spline_func=spline_func, norm_dist=norm_dist) -> float:
            val = np.exp(x) 
            return max(spline_func(val), 0.) * norm_dist.pdf(x)

        low, high = (m - 4 * stdev, m + 4 * stdev)
        disc_exp_payoff = disc * trapz(
            np.vectorize(integr_func0)(np.linspace(low, high, sample_points)),
            dx=(high - low) / (sample_points - 1)
        ) / (norm_dist.cdf(high) - norm_dist.cdf(low))
        return max(self.payoff(self.spot), disc_exp_payoff)



In [54]:
# Validate the grid_based algorithm against Black-Scholes formula for Europeans
optionGrid = GridAmerican(typ="call", spot=100.0, strike=120.0, T=3, t=0.5, r=0.05, sigma=0.2, num_dt=3, num_dx=100)
print(optionGrid.get_price)
optionBS = EuroPricing(typ="call", spot=100.0, strike=120.0, T=3, t=0.5, r=0.05, sigma=0.2)
print(optionBS.get_price())

12.385804371107985
10.194150042249355


### Implement Longstaff-Schwartz Algorithm and ensure it validates against binary tree/grid-based solution for path-independent options

In [47]:
class LS_American(EuroPricing):
    '''
    Longstaff-Schwartz Algorithm for American Option Pricing
    '''
    def __init__(self, typ: str, spot: float, strike: float, T: float, t: float, r: float, sigma: float, 
                 num_laguerre_val: int, num_paths: int, num_dt: int):
        super().__init__(typ, spot, strike, T, t, r, sigma)
        self.payoff = lambda x: self.vanilla_american_payoff(x)
        self.ir = lambda tt: self.r * tt  # interest rate with time
        self.isig = lambda tt: (self.sigma ** 2) * tt # volatility with time

        self.feature_funcs = [lambda _, x: 1.] + [(lambda _, x, i=i: laguerre_feature_func(x[-1], i)) for i in
                               range(num_laguerre_val)]
        self.get_ls_price = self.get_lsprice(num_paths, num_dt)
        
    def vanilla_american_payoff(self, x: np.ndarray):
        if self.call:
            return max(x[-1] - self.strike, 0.)
        else:
            return max(self.strike - x[-1], 0.)
        
    def get_price_mean_var(self, x: float, t: float, dt: float):
        # get the mean and variance of future price given the current price 
        ir_t = self.ir(t)
        ir_t1 = self.ir(t + dt)
        isig_t = self.isig(t)
        isig_t1 = self.isig(t + dt)
        ir_diff = ir_t1 - ir_t
        isig_diff = isig_t1 - isig_t
        mean = np.log(x) + ir_diff - isig_diff / 2.
        var = isig_diff
        return mean, var

    def get_all_paths(self, num_paths: int, num_dt: int):
        # randomly generate Monte-Carlo price paths 
        dt = self.T / num_dt
        paths = np.empty([num_paths, num_dt + 1])
        paths[:, 0] = self.spot
        for i in range(num_paths):
            price = self.spot
            for t in range(num_dt):
                m, v = self.get_price_mean_var(price, t, dt)
                temp = np.random.normal(m, np.sqrt(v)) # from a normal distribution with future price mean and variance
                price = np.exp(temp)
                paths[i, t + 1] = price
        return paths          

    def get_lsprice(self, num_paths: int, num_dt: int):
        # path independent Longstaff-Schwartz Algorithm
        paths = self.get_all_paths(num_paths, num_dt)
        cashflow = np.array([max(self.payoff(paths[i, :]), 0.) for i in range(num_paths)])
        dt = self.T / num_dt
        for t in range(num_dt - 1, 0, -1):
            discount = np.exp(self.ir(t) - self.ir(t + dt))
            cashflow *= discount
            payoff = np.array([self.payoff(paths[i, :(t + 1)]) for i in range(num_paths)])
            indices = [i for i in range(num_paths) if payoff[i] > 0]
            if len(indices) > 0:
                # collect features for each path where payoff is positive
                x_vals = np.array([[f(t, paths[i, :(t + 1)]) for f in self.feature_funcs] for i in indices])
                y_vals = np.array([cashflow[i] for i in indices])
                # linear regression Y ~ X
                estimate = x_vals.dot(np.linalg.lstsq(x_vals, y_vals, rcond=None)[0])
                # compare the predicted value with the exercise price
                for i, ind in enumerate(indices):
                    if payoff[ind] > estimate[i]:
                        cashflow[ind] = payoff[ind]
        # return max(exercise, continue)
        return max(self.payoff(np.array([self.spot])),np.average(cashflow * np.exp(-self.ir(dt))))



In [48]:
# validate Longstaff-Schwartz pricing against Black-Scholes formula and grid_based pricing

strike_val = 80.0
num_laguerre_val = 3

def laguerre_feature_func(x: float,i: int) -> float:   
    xp = x / strike_val
    ident = np.eye(num_laguerre_val)
    return np.exp(-xp / 2) * lagval(xp, ident[i])

option = LS_American(typ="put", spot=100.0, strike=80.0, T=0.5, t=0, r=0.05, sigma=0.25, 
                     num_laguerre_val=3, num_paths = 20000, num_dt = 10)
optionGrid = GridAmerican(typ="put", spot=100.0, strike=80.0, T=0.5, t=2, r=0.05, sigma=0.25, num_dt=10, num_dx=100)
print("Black-Scholes price", option.get_price())
print("Longstaff-Schwartz price", option.get_ls_price)
print("Grid based price", optionGrid.get_price)

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in sqrt


Black-Scholes price 0.5663384123066049
Longstaff-Schwartz price 0.577935217702427
Grid based price 0.573837474097861
